<a href="https://colab.research.google.com/github/rikdantas/Aprendizagem-de-Maquinas/blob/main/ECT3695/Gradio/Classificando_Digitos_com_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificando digitos com Gradio
Nesse notebook, irei desenvolver uma aplicação que vai receber uma imagem de um dígito (entre 0 e 9) e um modelo irá classificar esse número. Para isso irei usar um modelo já treinado com o dataset de imagens MNIST e também irei utilizar o Gradio para criar uma interface.

## Instalando o Gradio e importando as bibliotecas

In [1]:
# Instalando o gradio
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 6.8 MB/s eta 0:00:00


In [31]:
# Importando as bibliotecas
import numpy as np
import tensorflow as tf
import gradio as gr
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import gdown
import cv2

## Importando o modelo

In [3]:
# Baixando o modelo
url = 'https://drive.google.com/uc?id=1UI599OPzFXMMiqo-CijF7VGyWua8K5o8'
output = 'mlp_mnist_model.h5'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1UI599OPzFXMMiqo-CijF7VGyWua8K5o8
To: /content/mlp_mnist_model.h5
100%|██████████| 1.34M/1.34M [00:00<00:00, 14.7MB/s]


'mlp_mnist_model.h5'

In [34]:
# Carregando o modelo
model = load_model('mlp_mnist_model.h5')

## Criando a interface com Gradio
Irei utilizar a biblioteca Gradio para criar uma interface onde o usuário vai desenhar um dígito. Depois iremos fazer um pré-processamento na imagem para ela ficar semelhante as que foram utilizadas para treinar o modelo. Como saída da interface iremos ter a imagem pré-processada e o rótulo previsto pelo modelo.

In [32]:
# Função para pré-processar a imagem e fazer a predição
def preprocess_and_predict(data):
    # Acessa a camada 'composite' do dicionário retornado pelo Sketchpad
    image = data.get("composite", None)
    if image is None:
        return "Erro: Imagem 'composite' não encontrada.", None

    # Converte a imagem de RGBA para RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)

    # Cria uma máscara para as áreas transparentes e preenche com branco
    alpha_channel = image[:, :, 3]  # Extrai o canal alfa
    _, mask = cv2.threshold(alpha_channel, 0, 255, cv2.THRESH_BINARY)
    white_background = np.ones_like(image_rgb, dtype=np.uint8) * 255
    image_rgb = np.where(mask[:, :, None] == 255, image_rgb, white_background)

    # Converte para escala de cinza
    gray_image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)

    # Inverte as cores para que o traço fique branco e o fundo preto
    inverted_image = cv2.bitwise_not(gray_image)

    # Redimensiona para 28x28
    resized_image = cv2.resize(inverted_image, (28, 28), interpolation=cv2.INTER_AREA)

    # Normaliza a imagem para o modelo (caso necessário)
    normalized_image = resized_image / 255.0  # Normaliza os valores para [0, 1]
    input_image = normalized_image.reshape(1, 28, 28, 1)  # Adiciona dimensões para o modelo

    # Faz a predição
    prediction = model.predict(input_image)
    label = np.argmax(prediction)  # Obtém o rótulo previsto

    return resized_image, str(label)  # Converte label para string

# Interface Gradio
iface = gr.Interface(
    fn=preprocess_and_predict,
    inputs=gr.Sketchpad(label="Desenhe um número", height=200, width=200),
    outputs=[
        gr.Image(label="Imagem Pré-processada (28x28)"),
        gr.Label(label="Dígito Previsto")
    ],
    title="Reconhecimento de Dígitos com Pré-processamento de Imagem",
    description="Desenhe um número e veja a imagem pré-processada junto com o rótulo previsto."
)

# Executa a interface
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a00f98d232ab99755b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


O resultado da previsão do modelo não mostrou uma acurácia muito grande, mas foi uma boa maneira para começar a usar a biblioteca Gradio.